In [1]:
import numpy as np
import pandas as pd
import nltk
from bs4 import BeautifulSoup
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3

In [44]:
titles = open("C:/Users/Saba Naseem/Desktop/Machine Learning/Codes/kaggle/cluster docs/document_cluster-master/title_list.txt").read().split('\n')

links = open("C:/Users/Saba Naseem/Desktop/Machine Learning/Codes/kaggle/cluster docs/document_cluster-master/link_list_imdb.txt").read().split('\n')

genres = open('C:/Users/Saba Naseem/Desktop/Machine Learning/Codes/kaggle/cluster docs/document_cluster-master/genres_list.txt').read().split('\n')

syn = open("C:/Users/Saba Naseem/Desktop/Machine Learning/Codes/kaggle/cluster docs/document_cluster-master/synopses_list_imdb.txt").read().split('\n BREAKS HERE')

In [45]:
genres[:3]

["[u' Crime', u' Drama']",
 "[u' Crime', u' Drama']",
 "[u' Biography', u' Drama', u' History']"]

In [47]:
links[:3]

['http://www.imdb.com/title/tt0068646/',
 'http://www.imdb.com/title/tt0111161/',
 'http://www.imdb.com/title/tt0108052/']

In [48]:
titles[:3]

['The Godfather', 'The Shawshank Redemption', "Schindler's List"]

In [49]:
data=pd.read_csv("C://Users/Saba Naseem/Desktop/Machine Learning/Codes/kaggle/Document Clustering/IMDB-Movie-Data.csv")